In [1]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy

def update_ohlc(symbol):
    if (not os.path.isfile('data/'+symbol+'-ohlc.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), datetime.datetime.now())

    else:
        df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
        start = df_train.index[-1] + datetime.timedelta(days=1)
        print(start, datetime.datetime.now())

        if(start <= datetime.datetime.now()):
            new_data = data.DataReader(symbol, 'yahoo', start, datetime.datetime.now() )
            print(new_data)
            df_train = df_train.append(new_data)

    df_train.to_csv('data/'+symbol+'-ohlc.csv')
    
    if (not os.path.isfile('data/'+symbol+'-dividens.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), datetime.datetime.now())
symbols = ['spy', 'qqq','iwm', 'eem', 'aapl', 'ibm', 'fb','goog', 'amzn','gs','intc', 'dva',
           'noah', 'lc', 'renn', 'tsla','hog', '600519.SS']
#symbols = ['fcx']
for symbol in symbols:
    update_ohlc(symbol)
print("update completed.")

C:\Anaconda3\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
C:\Anaconda3\lib\site-packages\pandas\io\wb.py:21: FutureWarning: 
The pandas.io.wb module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


2016-04-08 00:00:00 2016-04-11 08:17:15.492389
                  Open        High         Low  Close    Volume  Adj Close
Date                                                                      
2016-04-08  205.339996  205.850006  203.869995  204.5  92431200      204.5
2016-04-08 00:00:00 2016-04-11 08:17:16.107847
                  Open        High         Low  Close    Volume  Adj Close
Date                                                                      
2016-04-08  109.769997  110.019997  108.620003  109.0  22745400      109.0
2016-04-08 00:00:00 2016-04-11 08:17:16.548897
                  Open    High         Low       Close    Volume   Adj Close
Date                                                                        
2016-04-08  109.610001  110.07  108.629997  109.110001  25463900  109.110001
2016-04-08 00:00:00 2016-04-11 08:17:17.077986
                 Open   High    Low      Close    Volume  Adj Close
Date                                                           